In [1]:
#embedding 테스트 - 완료
##one-hot으로 할 필요가 없음.
##embedding_size도 마음대로 결정할 수 있음.
##random init으로 해도됨.
##input의 embedding을 training할수도 있고 안할수도 있음
##Y는 embedding 할 필요 없음. 그냥 target class shape만 맞춰서 loss를 구하면 됨

#file에서 읽어와서 input으로 넣어보기

#padding 적용

from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility

In [2]:
sentences = ["그는 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다.",
            "김씨가 아버지에게 보내는 편지를 읽고 무대 뒤로 퇴장하려 하자 문 대통령은 자리에서 일어나 김씨를 따라가 안아주며 위로했다."]

char_set = list(set(' '.join(sentences)))
char_dic = {w: i for i, w in enumerate(char_set)}
print(char_dic)
# sentence = ("그는 이날 기념식에서 아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 낭독했다.")
# char_set = list(set(sentence))
# char_dic = {w: i for i, w in enumerate(char_set)}
# print(char_dic)

{'픈': 1, '따': 2, '주': 3, '식': 4, '김': 6, '위': 35, '념': 7, '아': 8, '문': 12, '했': 11, '무': 48, '버': 13, '안': 18, '그': 16, '며': 19, '낭': 20, '대': 21, '추': 5, '나': 22, '생': 23, '에': 24, '가': 25, '날': 14, '령': 26, '통': 27, '기': 28, '이': 29, '로': 30, '일': 15, '읽': 32, '뒤': 33, '사': 34, '장': 40, '목': 36, '은': 38, '게': 39, '치': 41, '.': 42, '하': 43, '보': 44, '어': 45, '는': 46, '의': 47, '를': 9, '편': 49, '바': 59, '퇴': 50, '’': 57, '씨': 51, '자': 52, '제': 53, '리': 54, '모': 55, ' ': 56, '서': 10, '독': 58, '다': 37, '슬': 31, '라': 0, '지': 60, '‘': 61, '고': 63, '내': 62, '려': 17}


In [3]:
chosung_sentences = ["ㄱㄴ ㅇㄴ ㄱㄴㅅㅇㅅ ㅇㅂㅈㅇㄱ ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ ㅊㅁㅅㄹ ㄴㄷㅎㄷ.",
                    "ㄱㅆㄱ ㅇㅂㅈㅇㄱ ㅂㄴㄴ ㅍㅈㄹ ㅇㄱ ㅁㄷ ㄷㄹ ㅌㅈㅎㄹ ㅎㅈ ㅁ ㄷㅌㄹㅇ ㅈㄹㅇㅅ ㅇㅇㄴ ㄱㅆㄹ ㄸㄹㄱ ㅇㅇㅈㅁ ㅇㄹㅎㄷ."]
chosung_set = list(set(' '.join(chosung_sentences)))
chosung_dic = {w: i for i, w in enumerate(chosung_set)}
print(chosung_dic)
# chosung_sentence = ("ㄱㄴ ㅇㄴ ㄱㄴㅅㅇㅅ ㅇㅂㅈㅇㄱ ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ ㅊㅁㅅㄹ ㄴㄷㅎㄷ.")
# chosung_set = list(set(chosung_sentence))
# chosung_dic = {w: i for i, w in enumerate(chosung_set)}
# print(chosung_dic)

{'.': 12, 'ㅌ': 0, 'ㄴ': 1, 'ㅁ': 7, 'ㅍ': 2, 'ㅈ': 5, 'ㅇ': 4, 'ㄱ': 18, '’': 8, 'ㅊ': 6, 'ㅂ': 10, ' ': 3, 'ㄹ': 13, 'ㅅ': 14, 'ㄸ': 15, 'ㅎ': 16, 'ㅆ': 9, 'ㄷ': 11, '‘': 17}


In [4]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
embedding_size = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.05

In [5]:
dataX = []
dataY = []
for j in range(len(sentences)):
    sentence = sentences[j]
    chosung_sentence = chosung_sentences[j]
    for i in range(0, len(sentence) - sequence_length):
        x_str = sentence[i:i + sequence_length]
        y_str = chosung_sentence[i:i + sequence_length]
        print(i, x_str, '->', y_str)

        x = [char_dic[c] for c in x_str]  # x str to index
        y = [chosung_dic[c] for c in y_str]  # y str to index

        dataX.append(x)
        dataY.append(y)

0 그는 이날 기념식에 -> ㄱㄴ ㅇㄴ ㄱㄴㅅㅇ
1 는 이날 기념식에서 -> ㄴ ㅇㄴ ㄱㄴㅅㅇㅅ
2  이날 기념식에서  ->  ㅇㄴ ㄱㄴㅅㅇㅅ 
3 이날 기념식에서 아 -> ㅇㄴ ㄱㄴㅅㅇㅅ ㅇ
4 날 기념식에서 아버 -> ㄴ ㄱㄴㅅㅇㅅ ㅇㅂ
5  기념식에서 아버지 ->  ㄱㄴㅅㅇㅅ ㅇㅂㅈ
6 기념식에서 아버지에 -> ㄱㄴㅅㅇㅅ ㅇㅂㅈㅇ
7 념식에서 아버지에게 -> ㄴㅅㅇㅅ ㅇㅂㅈㅇㄱ
8 식에서 아버지에게  -> ㅅㅇㅅ ㅇㅂㅈㅇㄱ 
9 에서 아버지에게 바 -> ㅇㅅ ㅇㅂㅈㅇㄱ ㅂ
10 서 아버지에게 바치 -> ㅅ ㅇㅂㅈㅇㄱ ㅂㅊ
11  아버지에게 바치는 ->  ㅇㅂㅈㅇㄱ ㅂㅊㄴ
12 아버지에게 바치는  -> ㅇㅂㅈㅇㄱ ㅂㅊㄴ 
13 버지에게 바치는 ‘ -> ㅂㅈㅇㄱ ㅂㅊㄴ ‘
14 지에게 바치는 ‘슬 -> ㅈㅇㄱ ㅂㅊㄴ ‘ㅅ
15 에게 바치는 ‘슬픈 -> ㅇㄱ ㅂㅊㄴ ‘ㅅㅍ
16 게 바치는 ‘슬픈  -> ㄱ ㅂㅊㄴ ‘ㅅㅍ 
17  바치는 ‘슬픈 생 ->  ㅂㅊㄴ ‘ㅅㅍ ㅅ
18 바치는 ‘슬픈 생일 -> ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ
19 치는 ‘슬픈 생일’ -> ㅊㄴ ‘ㅅㅍ ㅅㅇ’
20 는 ‘슬픈 생일’이 -> ㄴ ‘ㅅㅍ ㅅㅇ’ㅇ
21  ‘슬픈 생일’이라 ->  ‘ㅅㅍ ㅅㅇ’ㅇㄹ
22 ‘슬픈 생일’이라는 -> ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ
23 슬픈 생일’이라는  -> ㅅㅍ ㅅㅇ’ㅇㄹㄴ 
24 픈 생일’이라는 제 -> ㅍ ㅅㅇ’ㅇㄹㄴ ㅈ
25  생일’이라는 제목 ->  ㅅㅇ’ㅇㄹㄴ ㅈㅁ
26 생일’이라는 제목의 -> ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ
27 일’이라는 제목의  -> ㅇ’ㅇㄹㄴ ㅈㅁㅇ 
28 ’이라는 제목의 추 -> ’ㅇㄹㄴ ㅈㅁㅇ ㅊ
29 이라는 제목의 추모 -> ㅇㄹㄴ ㅈㅁㅇ ㅊㅁ
30 라는 제목의 추모사 -> ㄹㄴ ㅈㅁㅇ ㅊㅁㅅ
31 는 제목의 추모사를 -> ㄴ ㅈㅁㅇ ㅊㅁㅅㄹ
32  제목의 추모사를  ->  ㅈㅁㅇ ㅊㅁㅅㄹ 
33 제목의 추모사를 낭 -> ㅈㅁㅇ ㅊㅁㅅㄹ ㄴ
34 목의 추모사를 낭독 -> ㅁㅇ ㅊㅁㅅㄹ ㄴㄷ
35 의 추모사를 낭독했 -> ㅇ ㅊㅁㅅㄹ ㄴㄷㅎ
36

In [6]:
test_sentence = "아버지에게 바치는 ‘슬픈 생일’이라는 제목의 추모사를 그는 이날 기념식에서 낭독했다."
test_chosung_sentence = "ㅇㅂㅈㅇㄱ ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ ㅊㅁㅅㄹ ㄱㄴ ㅇㄴ ㄱㄴㅅㅇㅅ ㄴㄷㅎㄷ."

test_dataX = []
test_dataY = []
for i in range(0, len(test_sentence) - sequence_length):
    x_str = test_sentence[i:i + sequence_length]
    y_str = test_chosung_sentence[i:i + sequence_length]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [chosung_dic[c] for c in y_str]  # y str to index

    test_dataX.append(x)
    test_dataY.append(y)

0 아버지에게 바치는  -> ㅇㅂㅈㅇㄱ ㅂㅊㄴ 
1 버지에게 바치는 ‘ -> ㅂㅈㅇㄱ ㅂㅊㄴ ‘
2 지에게 바치는 ‘슬 -> ㅈㅇㄱ ㅂㅊㄴ ‘ㅅ
3 에게 바치는 ‘슬픈 -> ㅇㄱ ㅂㅊㄴ ‘ㅅㅍ
4 게 바치는 ‘슬픈  -> ㄱ ㅂㅊㄴ ‘ㅅㅍ 
5  바치는 ‘슬픈 생 ->  ㅂㅊㄴ ‘ㅅㅍ ㅅ
6 바치는 ‘슬픈 생일 -> ㅂㅊㄴ ‘ㅅㅍ ㅅㅇ
7 치는 ‘슬픈 생일’ -> ㅊㄴ ‘ㅅㅍ ㅅㅇ’
8 는 ‘슬픈 생일’이 -> ㄴ ‘ㅅㅍ ㅅㅇ’ㅇ
9  ‘슬픈 생일’이라 ->  ‘ㅅㅍ ㅅㅇ’ㅇㄹ
10 ‘슬픈 생일’이라는 -> ‘ㅅㅍ ㅅㅇ’ㅇㄹㄴ
11 슬픈 생일’이라는  -> ㅅㅍ ㅅㅇ’ㅇㄹㄴ 
12 픈 생일’이라는 제 -> ㅍ ㅅㅇ’ㅇㄹㄴ ㅈ
13  생일’이라는 제목 ->  ㅅㅇ’ㅇㄹㄴ ㅈㅁ
14 생일’이라는 제목의 -> ㅅㅇ’ㅇㄹㄴ ㅈㅁㅇ
15 일’이라는 제목의  -> ㅇ’ㅇㄹㄴ ㅈㅁㅇ 
16 ’이라는 제목의 추 -> ’ㅇㄹㄴ ㅈㅁㅇ ㅊ
17 이라는 제목의 추모 -> ㅇㄹㄴ ㅈㅁㅇ ㅊㅁ
18 라는 제목의 추모사 -> ㄹㄴ ㅈㅁㅇ ㅊㅁㅅ
19 는 제목의 추모사를 -> ㄴ ㅈㅁㅇ ㅊㅁㅅㄹ
20  제목의 추모사를  ->  ㅈㅁㅇ ㅊㅁㅅㄹ 
21 제목의 추모사를 그 -> ㅈㅁㅇ ㅊㅁㅅㄹ ㄱ
22 목의 추모사를 그는 -> ㅁㅇ ㅊㅁㅅㄹ ㄱㄴ
23 의 추모사를 그는  -> ㅇ ㅊㅁㅅㄹ ㄱㄴ 
24  추모사를 그는 이 ->  ㅊㅁㅅㄹ ㄱㄴ ㅇ
25 추모사를 그는 이날 -> ㅊㅁㅅㄹ ㄱㄴ ㅇㄴ
26 모사를 그는 이날  -> ㅁㅅㄹ ㄱㄴ ㅇㄴ 
27 사를 그는 이날 기 -> ㅅㄹ ㄱㄴ ㅇㄴ ㄱ
28 를 그는 이날 기념 -> ㄹ ㄱㄴ ㅇㄴ ㄱㄴ
29  그는 이날 기념식 ->  ㄱㄴ ㅇㄴ ㄱㄴㅅ
30 그는 이날 기념식에 -> ㄱㄴ ㅇㄴ ㄱㄴㅅㅇ
31 는 이날 기념식에서 -> ㄴ ㅇㄴ ㄱㄴㅅㅇㅅ
32  이날 기념식에서  ->  ㅇㄴ ㄱㄴㅅㅇㅅ 
33 이날 기념식에서 낭 -> ㅇㄴ ㄱㄴㅅㅇㅅ ㄴ
34 날 기념식에서 낭독 -> ㄴ ㄱㄴㅅㅇㅅ ㄴㄷ
35  기념식에서 낭독했 ->  ㄱㄴㅅㅇㅅ ㄴㄷㅎ
36

In [7]:
batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
# X_one_hot = tf.one_hot(X, embedding_size)
# print(X_one_hot)  # check out the shape

In [8]:
print(num_classes)
embeddings = tf.Variable(
    tf.random_uniform([num_classes, 1], -1.0, 1.0), trainable=False)
X_one_hot = tf.nn.embedding_lookup(embeddings, X)
print(X_one_hot)

64
Tensor("embedding_lookup:0", shape=(?, 10, 1), dtype=float32)


In [9]:
# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, X_one_hot, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataY, Y: dataX})
    if i % 10 == 1:
        print('step:',i,' loss:',l)
        embds, results = sess.run([embeddings,outputs], feed_dict={X: dataY})
        print(embds)
        for j, result in enumerate(results):
            index = np.argmax(result, axis=1)
            if j is 0:  # print all for the first result to make a sentence
                print(''.join([char_set[t] for t in index]), end='')
            else:
                print(char_set[index[-1]], end='')
        print("\n\n")

step: 1  loss: 3.94641
[[ 0.15111303]
 [-0.72339869]
 [-0.08999562]
 [ 0.4218123 ]
 [-0.38752127]
 [ 0.93685055]
 [ 0.17448449]
 [-0.26229143]
 [-0.82632494]
 [ 0.59733224]
 [-0.78892827]
 [-0.94038844]
 [ 0.2947576 ]
 [ 0.7781589 ]
 [-0.09943175]
 [-0.65684891]
 [ 0.02457833]
 [ 0.28656936]
 [ 0.55926347]
 [ 0.00767303]
 [-0.59772944]
 [-0.33964396]
 [-0.55341482]
 [ 0.7954011 ]
 [ 0.0722971 ]
 [ 0.67704415]
 [ 0.42662692]
 [ 0.3120985 ]
 [-0.65985823]
 [ 0.70553827]
 [-0.8182025 ]
 [-0.29565001]
 [ 0.70450878]
 [ 0.72135234]
 [ 0.12154818]
 [-0.91761923]
 [-0.63982415]
 [ 0.8215034 ]
 [ 0.05097055]
 [ 0.77491093]
 [-0.66267157]
 [-0.53732491]
 [ 0.47249365]
 [-0.44757557]
 [-0.14903283]
 [-0.79435015]
 [-0.47362828]
 [-0.70770597]
 [-0.7509048 ]
 [-0.11077881]
 [ 0.31454897]
 [-0.88344765]
 [ 0.51833987]
 [ 0.20747089]
 [ 0.49172091]
 [-0.32318234]
 [ 0.16677785]
 [ 0.32375574]
 [-0.02925086]
 [-0.77026868]
 [ 0.57969475]
 [-0.25194216]
 [ 0.95253921]
 [ 0.08009982]]
                

In [11]:
#embedding 연습!!
import tensorflow as tf

embeddings = tf.Variable(
    tf.random_uniform([num_classes, 50], -1.0, 1.0))

train_inputs = tf.placeholder(tf.int32, shape=[3,2])
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
ems, inputs,embd = sess.run([embeddings,train_inputs,embed], feed_dict={train_inputs:[[1,2],[2,3],[3,4]]})

for i in ems:
    print(i)
print('-----------')
print(inputs)
print('-----------')
print(embd)

[ 0.03279352  0.3813386  -0.87872863  0.73162794 -0.29813266]
[ 0.07190323  0.47247267  0.33953214  0.451756    0.48886824]
[-0.37702703 -0.89289188 -0.0033505  -0.48603177 -0.47505999]
[ 0.18791056  0.68445444  0.85997272 -0.23586679 -0.15762901]
[ 0.78887939  0.28453135  0.9487586   0.73153853  0.44170451]
[-0.30867434  0.24548388  0.40597773  0.30109787  0.50034475]
[ 0.49145913  0.5308435   0.01785254 -0.85992765  0.57344198]
[ 0.99143147 -0.80973268 -0.26515603  0.58108091 -0.06477666]
[-0.65864086  0.06885624  0.95042109  0.25653911 -0.67929101]
[-0.42458677 -0.08251071 -0.13261127  0.23038697  0.40781975]
-----------
[[1 2]
 [2 3]
 [3 4]]
-----------
[[[ 0.07190323  0.47247267  0.33953214  0.451756    0.48886824]
  [-0.37702703 -0.89289188 -0.0033505  -0.48603177 -0.47505999]]

 [[-0.37702703 -0.89289188 -0.0033505  -0.48603177 -0.47505999]
  [ 0.18791056  0.68445444  0.85997272 -0.23586679 -0.15762901]]

 [[ 0.18791056  0.68445444  0.85997272 -0.23586679 -0.15762901]
  [ 0.7888